01. 데이터 및 라이브러리 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [199]:
# 라이브러리 불러오기
import random
import statistics
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_similarity

In [185]:
# 데이터 불러오기
# user = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Alcohol_Data/USER_TEST.csv')
user = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Alcohol_Data/전통주 데이터셋 - USER.csv')
input = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Alcohol_Data/INPUT.csv')
output = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Alcohol_Data/OUTPUT.csv')

02. 원본데이터 가공하기

In [ ]:
# 원본데이터 수정
rating = user[['unique_id', 'golajum', 'rate']]
rating

3. 데이터 전처리

In [ ]:
# rate 평점이 없는 값을 0으로 변경
rating['rate'] = rating['rate'].fillna(0)
rating

4. 피벗테이블 만들기

In [ ]:
# 피벗테이블 만들기 - user별 술을 평가한 평점 데이터에 대한 pivot table 제작
rating_matrix = rating.pivot_table(index = 'unique_id', columns = 'golajum', values = 'rate')

# 결측치 지우기
rating_matrix = rating_matrix.fillna(0)
rating_matrix

5. 비슷한 성향의 유저 찾기 - 코사인 유사도

- user_id: 현재 사용자
- matrix: 전통주에 대한 사용자 평점 데이터를 포함하는 피벗 테이블
- k: 반환할 상위 유사 유저의 수

In [189]:
def similar_users(user_id, matrix, k=5):

    # 사용자의 데이터 추출
    user_data = matrix[matrix.index == user_id]

    # 다른 모든 사용자의 데이터 추출
    other_user_data = matrix[matrix.index != user_id]

    # 코사인 유사도 계산 및 리스트로 변환
    similarities = cosine_similarity(user_data, other_user_data)[0].tolist()

    # 사용자 유사도를 딕셔너리로 변환
    user_similarity = dict(zip(other_user_data.index, similarities))

    # 유사도를 기준으로 정렬하여 상위 k개의 사용자 선택
    user_similarity_sorted = sorted(user_similarity.items(), key=lambda x: x[1], reverse=True)
    top_user_similarities = user_similarity_sorted[:k]

    # # 설명을 위한 Print문 출력
    # print('1. 사용자의 데이터')
    # print(user_data)
    # print('2. 다른 사용자의 데이터')
    # print(other_user_data)
    # print('3. 다른 유저들과의 코사인 유사도')
    # print(similarities)
    # print('4. 유사도를 딕셔너리 형태로 변환')
    # print(user_similarity)
    # print('5, 가장 유사한 k명의 아이디와 유사도')
    # print(top_user_similarities)

    # 선택된 사용자들의 인덱스 반환
    users = [i[0] for i in top_user_similarities]
    return users

6. 전통주 추천하기
- user_id: 현재 사용자
- similar_user_indices: 유사한 사용자들의 인덱스 리스트
- matrix: 전통주에 대한 사용자 평점 데이터를 포함하는 피벗 테이블
- items: 추천할 전통주의 개수 (기본값은 1)

In [ ]:
def recommend_item(user_id, similar_user_indices, matrix, items=1):

    # 유사한 사용자들의 데이터 가져오기
    similar_users_data = matrix[matrix.index.isin(similar_user_indices)]
    print('==========================================')
    print(similar_users_data)

    # 비슷한 유저 평균 계산
    similar_users_data_mean = similar_users_data.mean(axis=0)
    print('==========================================')
    print(similar_users_data_mean)

    # 데이터 프레임 변환
    similar_users_df = pd.DataFrame(similar_users_data_mean, columns=['user_similarity'])
    print('==========================================')
    print(similar_users_df)

    # 현재 사용자의 데이터 가져오기
    user_data = matrix[matrix.index == user_id]
    print('==========================================')
    print(user_data)

    # 현재 사용자의 평가 데이터 정렬
    user_data_transposed = user_data.transpose()
    print('==========================================')
    print(user_data_transposed)

    # 컬럼명 변경
    user_data_transposed.columns = ['rating']

    # 미추천 전통주 목록만 뽑기
    user_data_transposed = user_data_transposed[user_data_transposed['rating'] == 0]
    print('==========================================')
    print(user_data_transposed)

    # 미추천 전통주 리스트 만들기
    unrated_golajums = user_data_transposed.index.tolist()
    print('==========================================')
    print(unrated_golajums)

    # 데이터프레임에서 미평가 전통주만 필터링
    similar_users_df_filtered = similar_users_df[similar_users_df.index.isin(unrated_golajums)]
    print('==========================================')
    print(similar_users_df_filtered)

    # 미추천 전통주를 기준으로 내림차순 정렬
    similar_users_df_ordered = similar_users_df_filtered.sort_values(by=['user_similarity'], ascending=False)
    print('==========================================')
    print(similar_users_df_ordered)

    # 상위 items개의 값 가져오기
    top_n_golajums = similar_users_df_ordered.head(items)
    print('==========================================')
    print(top_n_golajums)

    # 선택된 전통주의 이름을 가져오기
    top_n_golajums_indices = top_n_golajums.index.tolist()
    print('==========================================')
    print(top_n_golajums_indices)

    # 해당 전통주의 정보를 Output 데이터에서 불러오기
    recommended_golajums_information = output[output['상품명'].isin(top_n_golajums_indices)]
    print('==========================================')
    print(recommended_golajums_information)

    # 선택된 전통주의 정보를 반환
    return recommended_golajums_information

8. 알고리즘 적용

In [ ]:
# 사용자 기반 협업 필터링을 통한 추천 실행
user_to_recommend = 1
similar_users_indices = similar_users(user_to_recommend, rating_matrix, k=5)
recommended_items = recommend_item(user_to_recommend, similar_users_indices, rating_matrix, items=1)

# 결과 출력
print(recommended_items['상품명'])

ALS 기반 추천 시스템

In [206]:
import pandas as pd
import implicit
from scipy.sparse import csr_matrix

# 데이터 불러오기
user = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Alcohol_Data/전통주 데이터셋 - USER.csv')
input_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Alcohol_Data/INPUT.csv')
output_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Alcohol_Data/OUTPUT.csv')

# rate 평점이 없는 값을 0으로 변경
user['rate'] = user['rate'].fillna(0)

# 피벗테이블 만들기 - user별 술을 평가한 평점 데이터에 대한 pivot table 제작
rating_matrix = user.pivot_table(index='unique_id', columns='golajum', values='rate', fill_value=0)

# 사용자-아이템 행렬을 CSR(Compressed Sparse Row)로 변환
user_item_csr = csr_matrix(rating_matrix)

# ALS 모델 훈련
model = implicit.als.AlternatingLeastSquares(factors=50, regularization=0.1, iterations=50)
model.fit(user_item_csr)

def recommend_item_als(user_id, matrix, items=1):

    # 특정 사용자의 정보를 가지고 있는 행 추출
    user_data = matrix.loc[[user_id]]

    # 사용자 행을 CSR(Compressed Sparse Row) 행렬로 변환
    user_data_csr = csr_matrix(user_data)

    # ALS 모델을 사용하여 추천 생성
    recommended_item_indices, _ = model.recommend(user_id, user_data_csr, N=items)
    print(recommended_item_indices)

    # 선택된 전통주의 정보를 Output 데이터에서 불러오기
    recommended_golajums_information = output_data[output_data['상품명'].isin(recommended_item_indices)]
    print(recommended_golajums_information)

    # 선택된 전통주의 정보를 반환
    return recommended_golajums_information

# 특정 사용자에 대한 ALS 기반 추천 생성
user_id_to_recommend = 1  # 원하는 사용자의 ID로 변경
recommended_items_als = recommend_item_als(user_id_to_recommend, rating_matrix, items=5)

# 출력
recommended_items_als

  0%|          | 0/50 [00:00<?, ?it/s]

[2 5 7 3 4]
Empty DataFrame
Columns: [상품명, 주종, 절대 도수, 상대 도수, 용량, 가격, 가성비, 맥주 가성비, 검색용 상품명, 판매처, 단맛, 바디감, 신맛, 청량감, 향의 강도, 향의 종류, 사진]
Index: []


,상품명,주종,절대 도수,상대 도수,용량,가격,가성비,맥주 가성비,검색용 상품명,판매처,단맛,바디감,신맛,청량감,향의 강도,향의 종류,사진
